# Task for Today  

***

## Campus Recruitment Salary Prediction  

Given *data about campus recruitment*, let's try to predict the **salary** offered to a given student.

We will use a linear regression model with L2 regularization to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Ridge

In [ ]:
data = pd.read_csv('../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')

In [ ]:
data

In [ ]:
data.info()

# Preprocessing

In [ ]:
data

In [ ]:
def binary_encode(df, column_dict):
    df = df.copy()
    for column, positive_value in column_dict.items():
        df[column] = df[column].apply(lambda x: 1 if x == positive_value else 0)
    return df

def onehot_encode(df, column_dict):
    df = df.copy()
    for column, prefix in column_dict.items():
        dummies = pd.get_dummies(df[column], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(column, axis=1)
    return df

In [ ]:
def preprocess_inputs(df):
    df = df.copy()
    
    # Encode categorical features
    binary_feature_dict = {
        'gender': 'M',
        'ssc_b': 'Central',
        'hsc_b': 'Central',
        'workex': 'Yes',
        'specialisation': 'Mkt&Fin',
        'status': 'Placed'
    }
    
    nominal_feature_dict = {
        'hsc_s': 'hsc',
        'degree_t': 'deg'
    }
    
    df = binary_encode(df, binary_feature_dict)
    df = onehot_encode(df, nominal_feature_dict)
    
    # Split missing salary data from df and save it for later
    missing_salaries = df[df.isna().sum(axis=1) > 0]
    missing_salaries = missing_salaries.drop('salary', axis=1)
    
    missing_salary_ids = missing_salaries['sl_no'].reset_index(drop=True).copy()
    
    df = df.drop(missing_salaries.index, axis=0).reset_index(drop=True)
    
    # Drop sl_no column
    df = df.drop('sl_no', axis=1)
    missing_salaries = missing_salaries.drop('sl_no', axis=1)
    
    # Split df into X and y
    y = df['salary'].copy()
    X = df.drop('salary', axis=1).copy()
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=123)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    missing_salaries = scaler.transform(missing_salaries)
    
    return X_train, X_test, y_train, y_test, missing_salaries, missing_salary_ids

In [ ]:
X_train, X_test, y_train, y_test, missing_salaries, missing_salary_ids = preprocess_inputs(data)

# Training/Results

In [ ]:
model = Ridge(alpha=100.0)
model.fit(X_train, y_train)

model_r2 = model.score(X_test, y_test)

print("Model R^2: {:.5f}".format(model_r2))

In [ ]:
missing_salary_predictions = pd.Series(model.predict(missing_salaries), name='salary')

print("Potential salaries for missing target values:")
pd.concat([missing_salary_ids, missing_salary_predictions], axis=1)

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/V0qZpYEAmww